# <span style="color:#ff1414"> BEDtools analysis. </span>

This is a script to answer research questions outlined elsewhere. In summary, this script:

1. compares methylation results between different methylation-callers, and between different methylation sequencing methods.

2. compares methylation between genes and non-gene regions

3. compares methylation between transposons and non-repetitive regions

4. compares transposons and genes


Note:
- PB/pb = PacBio
- ONT/ont = Oxford Nanopore Technology
- NP = Nanopolish

In [4]:
# load modules
import os
import glob
import pprint
import pandas as pd

In [52]:
#First we need to define the base dirs
DIRS = {}
DIRS['BASE2'] = '/home/anjuni/analysis'
DIRS['RNA'] = os.path.join(DIRS['BASE2'], 'rna_counts')
DIRS['MEAN_STD'] = os.path.join(DIRS['RNA'], 'average_and_stdev')
DIRS['TRIALS'] = os.path.join(DIRS['RNA'], 'trials_tsv')
DIRS['FIGURES'] = os.path.join(DIRS['BASE2'], 'figures')
DIRS['BLAST_FIG'] = os.path.join(DIRS['FIGURES'], 'blast_expression')

In [6]:
#Quick chech if directories exist
for value in DIRS.values():
    if not os.path.exists(value):
        print('%s does not exist' % value)

## <span style='color:#ffa347'> 10. Expression of methylation machinery throughout Pst life cycle. <span/>

##### candidate protein details copied from BLAST.ipnyb


#### DNMT1/MASC2

<span style='color:darkred'> query: XP_001833175.2, len = 1253 <span/>

<span style='color:purple'> h_subject: Pst104E_20230, len = 1248 <span/>

Score = 206 bits (523),  Expect = 5e-54, Method: Compositional matrix adjust.
Identities = 241/925 (26%), Positives = 372/925 (40%), Gaps = 178/925 (19%)


<span style='color:darkblue'> p_subject: Pst104E_04293, len = 1248 <span/>

Score = 206 bits (523),  Expect = 5e-54, Method: Compositional matrix adjust.
Identities = 241/925 (26%), Positives = 372/925 (40%), Gaps = 178/925 (19%)

#### RAD8

<span style='color:darkred'> query: XP_001831325.2, len = 2184 <span/>

<span style='color:purple'> h_subject: Pst104E_28179, len = 2204 <span/>

Score = 1158 bits (2996),  Expect = 0.0, Method: Compositional matrix adjust.
Identities = 643/1386 (46%), Positives = 837/1386 (60%), Gaps = 76/1386 (5%)



<span style='color:darkblue'> p_subject: Pst104E_12497, len = 2204 <span/>

Score = 1159 bits (2997),  Expect = 0.0, Method: Compositional matrix adjust.
Identities = 643/1386 (46%), Positives = 838/1386 (60%), Gaps = 76/1386 (5%)

DNMT1/MASC2
- h_subject: Pst104E_20230   //   gene_model_hcontig_0009_24.226
- p_subject: Pst104E_04293   //   gene_model_pcontig_009.363
    
RAD8
- h_subject: Pst104E_28179   //   gene_model_hcontig_0052_06.60
- p_subject: Pst104E_12497   //   gene_model_pcontig_052.64


In [7]:
# make dicts of BLAST results
blast_h = { 'DNMT1/MASC2' : 'gene_model_hcontig_0009_24.226', 'RAD8' : 'gene_model_hcontig_0052_06.60' }
blast_p = { 'DNMT1/MASC2' : 'gene_model_pcontig_009.363', 'RAD8' : 'gene_model_pcontig_052.64' }

In [18]:
# make dict of all BLAST results
blast_dict = { 'DNMT1/MASC2_h' : 'gene_model_hcontig_0009_24.226', 'RAD8_h' : 'gene_model_hcontig_0052_06.60', \
              'DNMT1/MASC2_p' : 'gene_model_pcontig_009.363', 'RAD8_p' : 'gene_model_pcontig_052.64' }

In [19]:
blast_dict

{'DNMT1/MASC2_h': 'gene_model_hcontig_0009_24.226',
 'DNMT1/MASC2_p': 'gene_model_pcontig_009.363',
 'RAD8_h': 'gene_model_hcontig_0052_06.60',
 'RAD8_p': 'gene_model_pcontig_052.64'}

In [ ]:
# make fn dict
rnaseq_fn_dict = {}
for fn in glob.iglob('%s/*_gene_rpkm_average.tsv' % DIRS['MEAN_STD'], recursive=True):
    if len(fn) == 87:
        rnaseq_fn_dict[fn.split('/')[-1]] = fn

rnaseq_fn_dict        

In [8]:
# make rnaseq df dict
rnaseq_df_dict = {}
for fn in glob.iglob('%s/*_gene_rpkm_average.tsv' % DIRS['MEAN_STD'], recursive=True):
    if len(fn) == 87:
        df = pd.read_csv(fn, header=0, sep='\t')
        rnaseq_df_dict[fn.split('/')[-1]] = df

print(*rnaseq_df_dict, sep='\t')

In [12]:
# check df format
rnaseq_df_dict['Pst_104E_v13_h_gene_rpkm_average.tsv'].head()

,gene_ID,GS,HE,IT0,IT6,IT9,UG
0,gene_model_hcontig_0000_03.1,0.487427,0.000000,0.000000,0.000000,2.377196,0.384531
1,gene_model_hcontig_0000_03.2,0.577405,144.208849,0.000000,1016.181477,456.791691,0.446980
2,gene_model_hcontig_0000_03.3,564.783136,52.086774,24.713948,235.779282,239.133947,316.475087
3,gene_model_hcontig_0000_03.4,0.000000,0.167865,0.000000,0.115347,0.179739,0.000000
4,EVM prediction%2hcontig_0000_003.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [51]:
# write function to pull line for each gene
def get_gene_expression(rnaseq_df, gene_name_dict):
    """This function makes a list of dataframes with expression data for the provided dictionary of genes."""
    df_list = []
    for key, value in gene_name_dict.items():
        df = rnaseq_df.loc[rnaseq_df['gene_ID'] == value]
        df_list.append(df)
    return df_list

In [35]:
# make final df for p and h contig
h_exp_df = pd.concat(get_gene_expression(rnaseq_df_dict['Pst_104E_v13_h_gene_rpkm_average.tsv'], blast_h))
p_exp_df = pd.concat(get_gene_expression(rnaseq_df_dict['Pst_104E_v13_p_gene_rpkm_average.tsv'], blast_p))

In [36]:
# check df format
h_exp_df

,gene_ID,GS,HE,IT0,IT6,IT9,UG
5163,gene_model_hcontig_0009_24.226,14.044223,10.199002,0.00000,8.278785,13.479055,8.753725
14740,gene_model_hcontig_0052_06.60,25.553134,5.275929,4.24689,23.023202,43.303914,23.250372


In [37]:
# check df format
p_exp_df

,gene_ID,GS,HE,IT0,IT6,IT9,UG
5174,gene_model_pcontig_009.363,12.534978,9.157605,0.000000,7.680131,12.338550,7.733644
15076,gene_model_pcontig_052.64,22.889871,4.734031,3.784534,21.413790,39.808364,20.647280


In [56]:
# save out the files
def save_tsv(df, file_name):
    out_fn = os.path.join(DIRS['BLAST_FIG'], file_name)
    df.to_csv(out_fn, sep='\t', header=True, index=None)
    
save_tsv(h_exp_df, 'average_exp_h.tsv')
save_tsv(p_exp_df, 'average_exp_p.tsv')

In [41]:
# getting all values, not just averages as before

# make a dict of file paths
trials_fn_dict = {}
for fn in glob.iglob('%s/*_gene_repRpkmMatrix_featureCounts.tsv' % DIRS['TRIALS'], recursive=True):
    if len(fn) == 95:
        trials_fn_dict[fn.split('/')[-1]] = fn

In [42]:
trials_fn_dict

{'Pst_104E_v13_h_gene_repRpkmMatrix_featureCounts.tsv': '/home/anjuni/analysis/rna_counts/trials_tsv/Pst_104E_v13_h_gene_repRpkmMatrix_featureCounts.tsv',
 'Pst_104E_v13_p_gene_repRpkmMatrix_featureCounts.tsv': '/home/anjuni/analysis/rna_counts/trials_tsv/Pst_104E_v13_p_gene_repRpkmMatrix_featureCounts.tsv'}

In [45]:
# make trials df dict
trials_df_dict = {}
for fn in glob.iglob('%s/*_gene_repRpkmMatrix_featureCounts.tsv' % DIRS['TRIALS'], recursive=True):
    if len(fn) == 95:
        df = pd.read_csv(fn, header=0, sep='\t')
        trials_df_dict[fn.split('/')[-1]] = df

In [46]:
print(*trials_df_dict)

Pst_104E_v13_p_gene_repRpkmMatrix_featureCounts.tsv Pst_104E_v13_h_gene_repRpkmMatrix_featureCounts.tsv


In [48]:
# make final df for p and h contig
h_trials_df = pd.concat(get_gene_expression(trials_df_dict['Pst_104E_v13_h_gene_repRpkmMatrix_featureCounts.tsv'], blast_h))
p_trials_df = pd.concat(get_gene_expression(trials_df_dict['Pst_104E_v13_p_gene_repRpkmMatrix_featureCounts.tsv'], blast_p))

In [49]:
# check df format
h_trials_df

,gene_ID,GS_1,GS_2,GS_3,HE_1,HE_2,HE_3,IT0_1,IT0_2,IT0_3,IT6_1,IT6_2,IT6_3,IT9_1,IT9_2,IT9_3,UG_1,UG_2,UG_3
5163,gene_model_hcontig_0009_24.226,15.047157,13.500206,13.585307,10.544356,10.044541,10.008109,0.0,0.0,0.000000,8.746266,9.365785,6.724305,11.581538,13.042110,15.813519,9.449517,7.129993,9.681665
14740,gene_model_hcontig_0052_06.60,26.808662,22.569493,27.281248,4.773597,5.700367,5.353824,0.0,0.0,12.740669,25.804973,19.635157,23.629477,47.950864,55.108638,26.852241,24.057654,21.909625,23.783836


In [50]:
# check df format
p_trials_df

,gene_ID,GS_1,GS_2,GS_3,HE_1,HE_2,HE_3,IT0_1,IT0_2,IT0_3,IT6_1,IT6_2,IT6_3,IT9_1,IT9_2,IT9_3,UG_1,UG_2,UG_3
5174,gene_model_pcontig_009.363,13.419446,12.043734,12.141756,9.451506,9.031854,8.989456,0.0,0.0,0.000000,8.025845,8.834853,6.179695,10.581197,11.990989,14.443465,8.345580,6.290840,8.564513
15076,gene_model_pcontig_052.64,24.000664,20.215019,24.453928,4.285874,5.073785,4.842435,0.0,0.0,11.353601,23.691241,18.685592,21.864536,43.870896,50.861249,24.692945,21.394362,19.483626,21.063851


In [57]:
# save out the dataframes
save_tsv(h_trials_df, 'trials_exp_h.tsv')
save_tsv(p_trials_df, 'trials_exp_p.tsv')